In [1]:
!pip install aiohttp~=3.6.2

     |████████████████████████████████| 1.2 MB 946 kB/s eta 0:00:01
     |████████████████████████████████| 149 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 256 kB 1.6 MB/s eta 0:00:01


In [2]:
!pip install PyPubSub

     |████████████████████████████████| 61 kB 1.2 MB/s eta 0:00:01


In [3]:
!pip install asyncio

     |████████████████████████████████| 101 kB 1.3 MB/s ta 0:00:01


In [4]:
!python3 -m pip install --index-url https://test.pypi.org/simple/ aries_basic_controller

Looking in indexes: https://test.pypi.org/simple/


In [5]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = "/"
ADMIN_URL = "http://researcher-agent:8051"

In [6]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                           webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

In [7]:
# Spins up server to respond to any webhook events from the agent
import asyncio
asyncio.get_event_loop().run_until_complete(agent_controller.listen_webhooks())
# await agent_controller.listen_webhooks()

In [8]:
# Controllers re-emit webhooks as events through PyPubSub
# Basic state management is handled through defaults but you can choose to pass in custom handlers
custom_handler = []
agent_controller.register_listeners(custom_handler, defaults=True)

In [9]:
#Paste in invitation from data_owner agent
invite = {'connection_id': 'ca5f3332-e5d9-42c4-bad0-db8edc637fb4', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '7b485f18-2d8c-4d54-943a-016b623c33c0', 'recipientKeys': ['8b8rvN2SaxaSiCsgTh7qeU5c14y5QwEkDBCUtNDstStH'], 'label': 'Bob', 'serviceEndpoint': 'http://192.168.65.3:8020'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiN2I0ODVmMTgtMmQ4Yy00ZDU0LTk0M2EtMDE2YjYyM2MzM2MwIiwgInJlY2lwaWVudEtleXMiOiBbIjhiOHJ2TjJTYXhhU2lDc2dUaDdxZVU1YzE0eTVRd0VrREJDVXRORHN0U3RIIl0sICJsYWJlbCI6ICJCb2IiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly8xOTIuMTY4LjY1LjM6ODAyMCJ9'}

In [10]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
print(response)

ACCEPTED CONNECTION -  {'invitation_key': '8b8rvN2SaxaSiCsgTh7qeU5c14y5QwEkDBCUtNDstStH', 'their_label': 'Bob', 'state': 'request', 'initiator': 'external', 'request_id': '9eb467bb-dd71-467a-8e21-fffea970172a', 'routing_state': 'none', 'accept': 'manual', 'my_did': 'YLmMyavYq9rrZ9P6x6zDpF', 'invitation_mode': 'once', 'updated_at': '2020-06-24 13:28:47.682962Z', 'created_at': '2020-06-24 13:28:47.654059Z', 'connection_id': 'f51d599f-0d0f-4e48-83a4-7eed86a2ed37'}
{'invitation_key': '8b8rvN2SaxaSiCsgTh7qeU5c14y5QwEkDBCUtNDstStH', 'their_label': 'Bob', 'state': 'request', 'initiator': 'external', 'request_id': '9eb467bb-dd71-467a-8e21-fffea970172a', 'routing_state': 'none', 'accept': 'manual', 'my_did': 'YLmMyavYq9rrZ9P6x6zDpF', 'invitation_mode': 'once', 'updated_at': '2020-06-24 13:28:47.682962Z', 'created_at': '2020-06-24 13:28:47.654059Z', 'connection_id': 'f51d599f-0d0f-4e48-83a4-7eed86a2ed37'}


In [16]:
# Print out accepted Invite and Researcher ID
print("Researcher ID", response["connection_id"])
researcher_id = response["connection_id"]
print("Invite Accepted")
print(response)

Researcher ID 97838a70-6c54-4d50-a968-baf0f36de4cd
Invite Accepted
{'request_id': 'fb8ead9c-da32-4328-9a0e-541295078190', 'my_did': 'JmVHZsdrvdtMJjHE7WCi4q', 'routing_state': 'none', 'state': 'request', 'initiator': 'external', 'invitation_mode': 'once', 'accept': 'manual', 'connection_id': '97838a70-6c54-4d50-a968-baf0f36de4cd', 'created_at': '2020-06-23 14:13:43.720229Z', 'updated_at': '2020-06-23 14:13:43.877959Z', 'invitation_key': 'A74iU3TDpJc1P6f88osUnEFi8FzCMzbW9hvgYT6hDijs', 'their_label': 'Bob'}


In [ ]:
# Do a trust ping to ensure connection is accepted by data_owner
trust_ping = await agent_controller.connections.trust_ping(researcher_id,"hello")
print("TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA")
print(trust_ping)

print("RESEARCHER ID {} DATA ID {}".format(researcher_id,data_connection_id))

In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(researcher_id)
print("RESEARCH AGENT CONNECTION")
print(connection)

In [ ]:
#send some basic messages
message = await agent_controller.connections.send_message(researcher_id,"hello from researcher world!")
print("BASIC MESSAGE - RESEARCH -> DATA")
print(message)